# ACS Serial Stream Example - With No Known Device File

`acpype` assumes a default baudrate of 115200 bps for the ACS. 

Some convenience utilities are provided for finding a COM port that an ACS is attached to `acpype.utils.find_acs_port`. This is done by looking for the ACS packet registration packets over 1 second on an incomming stream. Note that this function will iterate through the available ports and will return the first available ACS port that is found. It is not guaranteed to work with multiple ACS instruments unless the port is first made unavailable.

In [3]:
from acpype import ACSStream, parse_packet
from acpype.utils import find_acs_port

## Retrieve a Single ACS Packet and Display the Parsed Data

In [4]:
port = find_acs_port()  # Find the port that has an ACS attached to it.
if port is not None: # If a port is found and it is streaming ACS registration bytes...
    print(f'ACS available on port {port}.\n')
    with ACSStream(port) as acss: # Open a serial object.
        while True:  # Continuously read the stream until a full packet is found.
            acss.read_stream()
            acs_packet = acss.find_packet()
            if acs_packet.packet_data is not None:
                print(f'Packet Byte Data: {acs_packet.packet_data}\n')
                parsed_packet = parse_packet(acs_packet)
                print(f'Packet acquired from {parsed_packet.serial_number}.\n')
                
                # Code to store parsed packet data would go here and you would remove the break statement below.
                break  # Break after finding the first full length packet, otherwise this example would continue forever.
                
    print(parsed_packet)
else:
    print('No ACS found on any port.')

ACS available on port COM5.

Packet Byte Data: bytearray(b'\xff\x00\xff\x00\x02\xc0\x05\x01S\x00\x00\x0b\x01\xd0\x00\x00\r\x9bq\x10\xab5\x01\xe1\x02\xcd\x00\x0c\x87\xc5\x01T\x02\x11\x01\x9a\x01\xfa\x01\xc1\x02[\x01\xd5\x02B\x02\x05\x02\xad\x02"\x02\x90\x02[\x03\x11\x02y\x02\xf7\x02\xc5\x03\x81\x02\xe0\x03f\x03<\x03\xfd\x03O\x03\xe7\x03\xc0\x04{\x03\xc6\x04g\x04O\x05\x14\x04V\x05\x01\x04\xfd\x05\xbb\x04\xf1\x05\xb0\x05\xc1\x06}\x05\xaa\x06v\x06\xaa\x07S\x06l\x07\\\x07\xa1\x08\x1e\x07&\x085\x08\x92\x08\xf0\x07\xf1\t\x10\t\x99\t\xd3\x08\xcb\n\x03\n\xbb\n\xd6\t\xc5\x0b\x13\x0c\t\x0b\xe2\n\xc1\x0c7\r]\x0c\xfc\x0b\xdb\ra\x0e\xdd\x0e7\x0c\xf9\x0e\xb6\x10h\x0fq\x0e6\x10\x05\x12!\x10\xec\x0f\xa5\x11\x9a\x14 \x12}\x11\x1a\x13N\x166\x14\x07\x12\x91\x15\x06\x18L\x15\xaa\x149\x16\xc2\x1a\xaf\x17h\x15\xd0\x18\xbb\x1d\x04\x19\x14\x17l\x1a\x8d\x1fe\x1a\xd3\x19\x11\x1cz!\xd8\x1ci\x1a\x93\x1eG$#\x1e\x15\x1cK \x0c&\xc0\x1f\xfd\x1e1"0)\xa5!\xdb\x1f\xf7$P,a#\x9a!\xbe&7/-%\x98#\xcc(_2b\'\xb1%\xed*\xc35\xb9*